### Importing Libraries

In [1]:
import numpy as np
from modeci_mdf.utils import simple_connect
from modeci_mdf.mdf import *
from modeci_mdf.execution_engine import EvaluableGraph
from modeci_mdf.utils import simple_connect
import graph_scheduler


### Importing MDF libraries

In [2]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras



### Importing helper functions

In [3]:
from Keras_MDF import *

###  Loading the model and printing out the layers 

In [40]:
new_model = tf.keras.models.load_model("model.h5")
for x in new_model.layers:
    print(x.name)



dense
dense_1
dense_2
dense_3


### Selective layers which will be used in MDF model

In [85]:
layers_to_extract = [ "dense", "dense_1", "dense_2"]

### Getting the weight and activation of layer

In [86]:
params, activations = get_weights_and_activation(layers_to_extract, new_model)


### Viewing the weights of the model

In [87]:
#View Weights of the Model
new_model.get_weights()


[array([[ 0.10398559, -0.52907103, -0.5763342 , -0.42851037, -0.04213673,
          0.13919373, -0.45565033, -0.12160711,  0.65372306, -0.44640994,
          0.54915607, -0.3764851 ],
        [-0.4265113 , -0.55738515,  0.44082814, -0.5989884 , -0.59382963,
          0.68513244, -0.18171296,  0.18477757,  0.36108476,  0.360938  ,
          0.6259903 , -0.28624615],
        [ 0.09505785,  0.09689969, -0.45767173,  0.1257245 ,  0.40503317,
          0.8595235 , -0.3624527 , -0.24859744,  0.36286658, -0.4621262 ,
         -0.29659566, -0.37829697],
        [ 0.44957379,  0.3258949 , -0.52011794,  0.07935911,  0.14579469,
          0.09530878, -0.5674388 , -0.56147987, -0.25999516,  0.34693575,
         -0.6629996 , -0.40959406]], dtype=float32),
 array([ 0.02978562,  0.        ,  0.        ,  0.        , -0.1130906 ,
         0.11993016,  0.        , -0.09276331, -0.12388226,  0.08502758,
         0.1135751 ,  0.        ], dtype=float32),
 array([[-0.18656065, -0.04099801, -0.4067576 ,  0

### Getting the weight and parameter

In [88]:
weights = new_model.layers[1].get_weights()[1]
bias = new_model.layers[0].get_weights()


### Fetching the model and graph from the python file


In [89]:
mod, mod_graph = init_model_with_graph("keras_to_MDF", "Keras_to_MDF_graph")

### input node

In [90]:
input_node = create_input_node("input_node", [2.0])
mod_graph.nodes.append(input_node)
print(mod_graph.to_yaml())


Keras_to_MDF_graph:
    nodes:
        input_node:
            parameters:
                input_node_in:
                    value:
                    - 2.0
            output_ports:
                input_node_out:
                    value: input_node_in



### Dense node

In [91]:
node = create_dense_node("dense_node", weights, bias)
mod_graph.nodes.append(node)
print(mod_graph.to_yaml())


Keras_to_MDF_graph:
    nodes:
        input_node:
            parameters:
                input_node_in:
                    value:
                    - 2.0
            output_ports:
                input_node_out:
                    value: input_node_in
        dense_node:
            input_ports:
                dense_node_in: {}
            parameters:
                wgts:
                    value:
                    - -0.11693787574768066
                    - 0.19391286373138428
                    - 0.0
                    - -0.16743193566799164
                    - -0.1985131800174713
                    - -0.002261135959997773
                    - -0.03576304763555527
                    - 0.14592917263507843
                bias:
                    value:
                    -   -   - 0.10398559272289276
                            - -0.529071033000946
                            - -0.5763341784477234
                            - -0.4285103678703308
                 

### Activation node

In [92]:
activation = create_activation_node("activation_node", "sigmoid")
mod_graph.nodes.append(activation)
print(mod_graph.to_yaml())



Keras_to_MDF_graph:
    nodes:
        input_node:
            parameters:
                input_node_in:
                    value:
                    - 2.0
            output_ports:
                input_node_out:
                    value: input_node_in
        dense_node:
            input_ports:
                dense_node_in: {}
            parameters:
                wgts:
                    value:
                    - -0.11693787574768066
                    - 0.19391286373138428
                    - 0.0
                    - -0.16743193566799164
                    - -0.1985131800174713
                    - -0.002261135959997773
                    - -0.03576304763555527
                    - 0.14592917263507843
                bias:
                    value:
                    -   -   - 0.10398559272289276
                            - -0.529071033000946
                            - -0.5763341784477234
                            - -0.4285103678703308
                 

In [93]:
eg = EvaluableGraph(mod_graph, verbose=False)


Init graph: Keras_to_MDF_graph


### Evaluating the current value of each of the nodes

In [94]:
eg.enodes["dense_node"].evaluable_outputs[f"{'dense_node'}_out"].curr_value

In [95]:
eg.enodes["input_node"].evaluable_outputs[f"{'input_node'}_out"].curr_value

In [96]:
eg.enodes["activation_node"].evaluable_outputs[f"{'activation_node'}_out"].curr_value

### Saving the model to file

In [97]:
mod.to_json_file("keras_to_MDF.json")


'keras_to_MDF.json'

In [98]:
mod.to_yaml_file("keras_to_MDF.yaml")


'keras_to_MDF.yaml'

### Generating the graph of the model

In [103]:

mod.to_graph_image(
        engine="dot",
        output_format="png",
        view_on_render=False,
        level=3,
        filename_root="keras_to_MDF",
        is_horizontal=True
    )

from IPython.display import Image
Image(filename="Keras_to_MDF_Example.png")


Converting MDF graph: Keras_to_MDF_graph to graphviz (level: 3, format: png)
    Node: input_node
    Node: dense_node


AttributeError: 'Parameter' object has no attribute 'shape'